In [2]:
import sympy as sy
import numpy as np

nondef1, nondef2, nondef3, nondef4, nondef5 = sy.symbols('nondef1, nondef2, nondef3, nondef4, nondef5')      

In [11]:
# Präfixe
_nano = 10 ** (-9)
_mikro = 10 ** (-6)
_milli = 10 ** (-3)
_centi = 10 ** (-2)
_dezi = 10 ** (-1)

_Kilo = 10 ** 3
_Mega = 10 ** 6
_Giga = 10 ** 9
_Tera = 10 ** 12

In [4]:
# feste Parameter und ihre Zahlenwerte: diese werden später in der Fehlerfunktion numerisch gemacht aber bleiben für die partiellen Ableitungen symbolisch
g, pi, mu_0, G = sy.symbols('g, pi, mu_0, G')            
                                                
fix_parameters = {
    G: 6.672 * 10 ** (-11),
    g: 9.81, 
    pi: 3.141,
    mu_0: 4 * np.pi * 10 **(-7),
}   

In [5]:
# verwendbare, symbolische Variablen, Zusätzliche einfach hinzufügen
t = sy.symbols('t')                     

In [6]:
def err_gauss(f, var1, x1, err1, var2=nondef1, x2=0, err2=0, var3=nondef2, x3=0, err3=0, var4=nondef3, x4=0, err4=0, var5=nondef4, x5=0, err5=0):
    f_x1 = sy.diff(f, var1)
    print(f'Ableitung nach {var1}: {f_x1}')   
    f_x1 = f_x1.subs({var2: x2, var3: x3, var4: x4, var5: x5})
    f_x2 = sy.diff(f, var2)
    if f_x2 != 0:
        print(f'Ableitung nach {var2}: {f_x2}')   
    f_x2 = f_x2.subs({var1: x1, var3: x3, var4: x4, var5: x5})
    f_x3 = sy.diff(f, var3)
    if f_x3 != 0:
        print(f'Ableitung nach {var3}: {f_x3}')   
    f_x3 = f_x3.subs({var1: x1, var2: x2, var4: x4, var5: x5})
    f_x4 = sy.diff(f, var4)
    if f_x4 != 0:
        print(f'Ableitung nach {var4}: {f_x4}')   
    f_x4 = f_x4.subs({var1: x1, var2: x2, var3: x3, var5: x5})
    f_x5 = sy.diff(f, var5)
    if f_x5 != 0:
        print(f'Ableitung nach {var5}: {f_x5}')   
    f_x5 = f_x5.subs({var1: x1, var2: x2, var3: x3, var4: x4})

    # feste paramter numerisch machen
    f = f.subs(fix_parameters)
    f_x1 = f_x1.subs(fix_parameters)
    f_x2 = f_x2.subs(fix_parameters)
    f_x3 = f_x3.subs(fix_parameters)
    f_x4 = f_x4.subs(fix_parameters)
    f_x5 = f_x5.subs(fix_parameters)

    value = f.subs({var1: x1, var2: x2, var3: x3, var4: x4, var5: x5})
    error = (err1**2*(f_x1.subs(var1, x1))**2+err2**2*(f_x2.subs(var2, x2))**2+err3**2*(f_x3.subs(var3, x3))**2+err4**2*(f_x4.subs(var4, x4))**2+err5**2*(f_x5.subs(var5, x5))**2)**0.5
    print(f'Wert: {value} +/- Fehler: {error}')
    return (value, error)

In [7]:
def gew_mittel(ar_val, ar_err):
    value_num = 0
    sigma_denom = 0
    for i in range(len(ar_val)):
        value_num += ar_val[i]/ar_err[i]**2
        sigma_denom += 1/ar_err[i]**2
    value = value_num/sigma_denom
    sigma = (1/sigma_denom)**0.5
    print(f'gew. Mittelwert: {value} +/- sigma: {sigma}')
    return (value, sigma)

In [8]:
# Beispielaufruf der Funktion zur Gauß'schen Fehlerrechnung
# Rückgabenwert: Tupel aus berechnetem Wert und seinem Fehler

err_gauss(
    g/2*t**2*pi**3,                 # Funktion: benötigt die oben definierten, symbolischen Variablen
    t, 10, 20,                      # Variable, Wert der Variable, Fehler der Variable
)

Ableitung nach t: g*pi**3*t
Wert: 15199.9731544005 +/- Fehler: 60799.8926176020


(15199.9731544005, 60799.8926176020)

In [21]:
# Rückgabewert: Tupel aus gewichteten Mittelwert und seinem Fehler
Delta_s_Acryl = abs(np.array([75 - 96, 75 - 99.7, 75 - 98.4, 75 - 99.5, 75 - 96.8]))*_centi
Delta_s_Wasser = abs(np.array([75 - 89.7, 75- 88.7, 75 - 87.4, 75 - 88.0, 75 - 89.1]))*_centi
Delta_s_err = np.array([0.7*_milli for i in range(len(Delta_s_Wasser))])

DAcryl = gew_mittel(
    Delta_s_Acryl,                       # Liste/Array mit Werten
    Delta_s_err                     # Liste/Array mit Fehlern der Werte
)


DWasser = gew_mittel(
    Delta_s_Wasser,                       # Liste/Array mit Werten
    Delta_s_err                     # Liste/Array mit Fehlern der Werte
)



gew. Mittelwert: 0.2308 +/- sigma: 0.00031304951684997057
gew. Mittelwert: 0.13580000000000003 +/- sigma: 0.00031304951684997057


In [13]:
np.sqrt(2)*0.05*_centi

0.0007071067811865476

In [28]:
v_l, l_Ac, l_Wa, DWa, DAc, vA, vW = sy.symbols('v_l, l_Ac, l_Wa, DWa, DAc, vA, vW')


v_Ac = err_gauss(
    v_l*l_Ac/(l_Ac+DAc),
    v_l, 0.28215*_Giga, 0.00008*_Giga,
    l_Ac, 49*_centi, 0.05*_centi,
    DAc, DAcryl[0], DAcryl[1],
)


v_Wa = err_gauss(
    v_l*l_Wa/(l_Wa+DWa),
    v_l, 0.28215*_Giga, 0.00008*_Giga,
    l_Wa, 50*_centi, 0.05*_centi,
    DWa, DWasser[0], DWasser[1],
)


err_gauss(
    v_l/vA,
    v_l, 0.28215*_Giga, 0.00008*_Giga,
    vA, v_Ac[0], v_Ac[1]
)


err_gauss(
    v_l/vA,
    v_l, 0.28215*_Giga, 0.00008*_Giga,
    vA, v_Wa[0], v_Wa[1]
)

Ableitung nach v_l: l_Ac/(DAc + l_Ac)
Ableitung nach l_Ac: -l_Ac*v_l/(DAc + l_Ac)**2 + v_l/(DAc + l_Ac)
Ableitung nach DAc: -l_Ac*v_l/(DAc + l_Ac)**2
Wert: 191805632.630411 +/- Fehler: 117577.404557602
Ableitung nach v_l: l_Wa/(DWa + l_Wa)
Ableitung nach l_Wa: -l_Wa*v_l/(DWa + l_Wa)**2 + v_l/(DWa + l_Wa)
Ableitung nach DWa: -l_Wa*v_l/(DWa + l_Wa)**2
Wert: 221885813.148789 +/- Fehler: 134683.572736910
Ableitung nach v_l: 1/vA
Ableitung nach vA: -v_l/vA**2
Wert: 1.47102040816327 +/- Fehler: 0.000993527911541529
Ableitung nach v_l: 1/vA
Ableitung nach vA: -v_l/vA**2
Wert: 1.27160000000000 +/- Fehler: 0.000851911451224326


(1.27160000000000, 0.000851911451224326)